### External downloads

Pretrained weights are a .pth file available for download on Dropbox.

### General config

In [ ]:
RESUME = '../../../../jihon-blob/sst300_0712_83000.pth' # path to weights
MOT_ROOT = '../../../data/mot' 
SAVE_FOLDER = '../../../data/sst/out'
LOG_FOLDER = '../../../data/sst/log'
BATCH_SIZE = 1
WRITE_FILE = True
TENSORBOARD = True
SAVE_COMBINE = False
TYPE = 'test'

### Model config

In [1]:
import sys
sys.path.insert(1, "../SST")

In [2]:
import pprint
from config.config import config

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(config)

use configure:  init_test_mot17
{   'alpha_valid': 0.8,
    'base_net': {   '1024': [],
                    '900': [   64,
                               64,
                               'M',
                               128,
                               128,
                               'M',
                               256,
                               256,
                               256,
                               'C',
                               512,
                               512,
                               512,
                               'M',
                               512,
                               512,
                               512]},
    'base_net_folder': '/home/ssm/ssj/weights/MOT17/vgg16_reducedfc.pth',
    'batch_size': 1,
    'cuda': True,
    'dataset_type': 'train',
    'default_mbox': {'1024': [], '900': [4, 6, 6, 6, 4, 4]},
    'detector': 'FRCNN',
    'extra_net': {   '1024': [],
                     '900': [   256,
  

In [3]:
from tracker import TrackerConfig

pp.pprint(TrackerConfig.__dict__)

mappingproxy({   '__dict__': <attribute '__dict__' of 'TrackerConfig' objects>,
                 '__doc__': None,
                 '__module__': 'tracker',
                 '__weakref__': <attribute '__weakref__' of 'TrackerConfig' objects>,
                 'cuda': True,
                 'decay': 0.995,
                 'get_all_choices': <staticmethod object at 0x7f53bf7ec890>,
                 'get_all_choices_decay': <staticmethod object at 0x7f53bf12e410>,
                 'get_all_choices_max_track_node': <staticmethod object at 0x7f53bf0cf290>,
                 'get_choices_age_node': <staticmethod object at 0x7f53bf0cf710>,
                 'get_configure_str': <staticmethod object at 0x7f53bf7ec690>,
                 'get_ua_choice': <staticmethod object at 0x7f53bf431f50>,
                 'image_size': (900, 900),
                 'max_bad_node': 0.9,
                 'max_draw_track_node': 30,
                 'max_object': 80,
                 'max_record_frame': 30,
     

### Setup

In [ ]:
from tracker import SSTTracker, TrackerConfig, Track
import cv2
from data.mot_data_reader import MOTDataReader
import numpy as np
from config.config import config
from utils.timer import Timer
import argparse
import os

### Run

In [ ]:
dataset_index = [1, 3, 6, 7, 8, 12, 14]
dataset_detection_type = {'-FRCNN', '-SDP', '-DPM'}

In [ ]:
dataset_image_folder_format = os.path.join(args.mot_root, args.type+'/MOT'+str(args.mot_version)+'-{:02}{}/img1')
detection_file_name_format=os.path.join(args.mot_root, args.type+'/MOT'+str(args.mot_version)+'-{:02}{}/det/det.txt')

if not os.path.exists(args.log_folder):
    os.mkdir(args.log_folder)

save_folder = ''
choice_str = ''
if not choice is None:
    choice_str = TrackerConfig.get_configure_str(choice)
    save_folder = os.path.join(args.log_folder, choice_str)
    if not os.path.exists(save_folder):
        os.mkdir(save_folder)

saved_file_name_format = os.path.join(save_folder, 'MOT'+str(args.mot_version)+'-{:02}{}.txt')
save_video_name_format = os.path.join(save_folder, 'MOT'+str(args.mot_version)+'-{:02}{}.avi')

In [ ]:
f = lambda format_str: [format_str.format(index, type) for type in dataset_detection_type for index in dataset_index]

In [ ]:
def test(choice=None):
    timer = Timer()
    for image_folder, detection_file_name, saved_file_name, save_video_name in zip(f(dataset_image_folder_format), f(detection_file_name_format), f(saved_file_name_format), f(save_video_name_format)):
        print('start processing '+saved_file_name)
        tracker = SSTTracker()
        reader = MOTDataReader(image_folder = image_folder,
                      detection_file_name =detection_file_name,
                               min_confidence=0.0)
        result = list()
        result_str = saved_file_name
        first_run = True
        for i, item in enumerate(reader):
            if i > len(reader):
                break

            if item is None:
                continue

            img = item[0]
            det = item[1]

            if img is None or det is None or len(det)==0:
                continue

            if len(det) > config['max_object']:
                det = det[:config['max_object'], :]

            h, w, _ = img.shape


            if first_run and args.save_video:
                vw = cv2.VideoWriter(save_video_name, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (w, h))
                first_run = False

            det[:, [2,4]] /= float(w)
            det[:, [3,5]] /= float(h)
            timer.tic()
            image_org = tracker.update(img, det[:, 2:6], args.show_image, i)
            timer.toc()
            print('{}:{}, {}, {}\r'.format(os.path.basename(saved_file_name), i, int(i*100/len(reader)), choice_str))
            if args.show_image and not image_org is None:
                cv2.imshow('res', image_org)
                cv2.waitKey(1)

            if args.save_video and not image_org is None:
                vw.write(image_org)

            # save result
            for t in tracker.tracks:
                n = t.nodes[-1]
                if t.age == 1:
                    b = n.get_box(tracker.frame_index-1, tracker.recorder)
                    result.append(
                        [i] + [t.id] + [b[0]*w, b[1]*h, b[2]*w, b[3]*h] + [-1, -1, -1, -1]
                    )
        # save data
        np.savetxt(saved_file_name, np.array(result).astype(int), fmt='%i')
        print(result_str)

    print(timer.total_time)
    print(timer.average_time)    

In [ ]:
all_choices = TrackerConfig.get_choices_age_node()
iteration = 3
# test()

i = 0
for age in range(1):
    for node in range(1):
        c = (0, 0, 4, 0, 3, 3)
        choice_str = TrackerConfig.get_configure_str(c)
        TrackerConfig.set_configure(c)
        print('=============================={}.{}=============================='.format(i, choice_str))
        test(c)
        i += 1